In [1]:
import pandas as pd

In [2]:
table00 = pd.read_json("data/merged-01-00", lines=True)
small = pd.read_json("data/merged-01-00-snippet", lines=True)

In [ ]:
# extras
table01 = pd.read_json("data/merged-01-01", lines=True)
table14 = pd.read_json("data/merged-02-14", lines=True)
table20 = pd.read_json("data/merged-03-20", lines=True)

In [3]:
pd.set_option('display.max_columns', 100)
small.head(10)

,_host,ad_network_id,ad_type,adlog_count,advertiser_id,bid_requests,bid_responses,c_cnt,c_flag_cnt,c_timestamp,campaign_id,campaign_type,ck,cr_cnt,creative_id,exp_mode,f_cnt,flag,geo_area_code,geo_city_code,geo_city_name,geo_continent_code,geo_country_code2,geo_country_code3,geo_dma_code,geo_postal_code,geo_region_name,geo_timezone,i_cnt,i_flag_cnt,i_timestamp,i_txn_fee,i_txn_rate,ip_address,is_bot,is_fraud,keywords,num_ads,pub_campaign_id,pub_network_id,r_cnt,r_num_ads_requested,r_num_ads_returned,r_num_ads_third_party,r_timestamp,rate_metric,referer,session_id,site_id,token,txn_fee,txn_rate,ua,ua_device,ua_device_type,ua_major,ua_minor,ua_name,ua_os,ua_os_name,url,user_agent,uuid,vi_cnt,vi_flag_cnt,vi_timestamp,vv_cnt,widget_id,zone_id
0,an-prod-ralphie-frontline-bulks.us-east-1,1349,[story],1,3587,[],[],0.0,0.0,NaN,20523,private,"{""widget_type"": [""text_feed""]}",1,1454941,NaN,0.0,NaN,203.0,5282901.0,Brookfield,NA,US,USA,501.0,06804,CT,America/New_York,1.0,0.0,2018-04-01T00:56:55.982000Z,NaN,NaN,68.116.200.34,NaN,False,,10.0,NaN,767,1,10,10,0,2018-04-01T00:56:55.194040Z,CPC,http%3A%2F%2Fwww.breitbart.com%2Fbig-journalis...,706a0d7116b840a194c81b49edbc0a29_f7ec539c,13042,6XKC5K21,NaN,NaN,Mozilla/5.0 (iPad; CPU OS 10_3_3 like Mac OS X...,iPad,TAB,10.0,0.0,Mobile Safari,iOS 10.3,iOS,http://media.breitbart.com/media/if/newsmax.html,Mozilla/5.0 (iPad; CPU OS 10_3_3 like Mac OS X...,60039c05-5285-49ca-b4eb-a435e9d8352b,0.0,0.0,NaN,0.0,NaN,26024
1,an-prod-ralphie-frontline-bedim.us-east-1,1349,[story],1,3587,[],[],0.0,0.0,NaN,20523,private,"{""widget_type"": [""text_feed""]}",1,1454941,NaN,0.0,NaN,205.0,4083697.0,Pinson,NA,US,USA,630.0,35126,AL,America/Chicago,1.0,0.0,2018-04-01T00:45:42.182000Z,NaN,NaN,174.223.128.183,NaN,False,,10.0,NaN,767,1,10,10,0,2018-04-01T00:45:41.834098Z,CPC,http%3A%2F%2Fwww.breitbart.com%2Fjerusalem%2F,8cc504de1a9f4424b21a4165261910cc_5123bfda,13042,6XKC5K21,NaN,NaN,Mozilla/5.0 (iPad; CPU OS 11_2_6 like Mac OS X...,iPad,TAB,11.0,0.0,Mobile Safari,iOS 11.2,iOS,http://media.breitbart.com/media/if/newsmax.html,Mozilla/5.0 (iPad; CPU OS 11_2_6 like Mac OS X...,504e4902-4a71-4155-9de8-32e4afc00013,0.0,0.0,NaN,0.0,NaN,26024
2,an-prod-ralphie-frontline-bedim.us-east-1,767,[story],1,4735,[],[],0.0,0.0,NaN,21827,outside,"{""widget_type"": [""text_feed""]}",1,1810968,NaN,0.0,NaN,901.0,4641239.0,Memphis,NA,US,USA,640.0,38117,TN,America/Chicago,1.0,0.0,2018-04-01T00:38:48.245000Z,0.0,0.0,75.65.54.41,NaN,False,,10.0,NaN,767,1,10,10,0,2018-04-01T00:38:42.196667Z,CPM,https%3A%2F%2Fwww.google.com%2F,0a84e86680984a909b077f5b0d90c2b4_7071cfdc,13182,0AIIPHOX,0.0,0.0,Mozilla/5.0 (iPad; CPU OS 11_2_6 like Mac OS X...,iPad,TAB,11.0,2.0,Mobile Safari,iOS 11.2,iOS,http://thehill.com/policy/national-security/38...,Mozilla/5.0 (iPad; CPU OS 11_2_6 like Mac OS X...,c182f217-aff0-4f43-b74c-8c19344efbbf,1.0,0.0,2018-04-01T00:39:06.102000Z,0.0,NaN,18310
3,an-prod-ralphie-frontline-goofs.us-east-1,727,[story],1,4909,[],[],NaN,NaN,NaN,26801,outside,,1,5639350,0.0,NaN,NaN,510.0,5355933.0,Hayward,NA,US,USA,807.0,94541,CA,America/Los_Angeles,NaN,NaN,NaN,NaN,NaN,172.58.38.241,NaN,False,"Pocket_SaveTopic_Month_finance,Pocket_SaveTopi...",NaN,NaN,727,1,1,1,0,2018-04-01T00:29:13.267027Z,CPM,,709d2c3baf6147fdba39a75a09cd1761_0611b821,10481,MPA9WFG3,NaN,NaN,Mozilla/5.0 (Linux; Android 8.0.0; Pixel 2 Bui...,Pixel 2,MOB,62.0,0.0,Chrome Mobile,Android 8.0,Android,http://getpocket.com,NaN,429TaA7bgeWM3pk04ed1783dc1p4g4e349bc6flc28o91b...,NaN,NaN,NaN,NaN,NaN,14414
4,an-prod-ralphie-frontline-soths.us-east-1,1349,[story],1,3587,[],[],0.0,0.0,NaN,20523,private,"{""widget_type"": [""text_feed""]}",1,1454941,NaN,0.0,NaN,701.0,5690532.0,Minot,NA,US,USA,687.0,58703,ND,America/Chicago,1.0,0.0,2018-04-01T00:11:24.602000Z,NaN,NaN,74.207.139.114,NaN,False,,10.0,NaN,767,1,10,10,0,2018-04-01T00:11:19.706421Z,CPC,http%3A%2F%2Fwww.breitbart.com%2F,b2d8ee6efb3445b191f895dfe03d52ac_5dcfde3a,13042,6XKC5K21,NaN,NaN,Mozilla/5.0 (Linux; Android 7.0; SAMSUNG SM-T8...,

In [4]:
from urllib.parse import unquote
from collections import defaultdict

def find_url_frq(df):
    ref_dict = defaultdict(lambda: 0, {})
    total = df.shape[0]
    for ref in df["referer"]:
        url = unquote(ref)
        index = url.find(".com")
        base_url = url[:index+len(".com")+1]
        ref_dict[base_url] += 1

    inorderkeys = sorted(ref_dict, key=lambda k: -ref_dict[k])
    cutoff = 5
    print("Finding top", cutoff, "referers:")
    for key in inorderkeys:
        if cutoff > 0:
            print(key, "----", ref_dict[key]/total)
            cutoff -= 1
        else:
            print("Done computing--------------")
            break
        
find_url_frq(table00)
# find_url_frq(table01)
# find_url_frq(table14)
# find_url_frq(table20)

Finding top 5 referers:
http://www.breitbart.com/ ---- 0.49450438877466485
 ---- 0.2932378994254512
http://www.armslist.com/ ---- 0.07269903772300242
http://thehill.com/ ---- 0.025547800246187285
https://www.google.com/ ---- 0.012440172129855407
Done computing--------------


In [43]:
from urllib.parse import unquote, urlparse
import re

def extract_parts(url):
    if url == None:
        url = ''
    o = urlparse(url)
    return o.netloc, o.path

def not_number(s):
    try:
        float(s)
        return False
    except ValueError:
        return True

def produce_keywords(path):
    MIN_LENGTH = 3
    BANNED_WORDS = ["html"]
    words = re.findall("[\w]+", path)
    filtered = [word for word in words if len(word) >= MIN_LENGTH and \
                word not in BANNED_WORDS and not_number(word)]
    return filtered
    
def append_keyword_cols(df):
    domains = {"url_domain": [], "ref_domain": []}
    keywords = {"url_keywords": [], "ref_keywords": []}
    
    for url in df["url"]:
        domain, path = extract_parts(url)
        words = produce_keywords(path)
        domains["url_domain"].append(domain)
        keywords["url_keywords"].append(words)
    for ref in df["referer"]:
        url = unquote(ref)
        domain, path = extract_parts(url)
        words = produce_keywords(path)
        domains["ref_domain"].append(domain)
        keywords["ref_keywords"].append(words)
    
    all_cols = {}
    all_cols.update(domains)
    all_cols.update(keywords)
    return pd.concat([df, pd.DataFrame(all_cols)], axis=1)

append_keyword_cols(table00)[["ref_domain", "ref_keywords", "url_domain", "url_keywords"]]

,ref_domain,ref_keywords,url_domain,url_keywords
0,www.breitbart.com,"[big, journalism, laura, ingraham, take, easte...",media.breitbart.com,"[media, newsmax]"
1,www.breitbart.com,[jerusalem],media.breitbart.com,"[media, newsmax]"
2,www.google.com,[],thehill.com,"[policy, national, security, trump, sessions, ..."
3,,[],getpocket.com,[]
4,www.breitbart.com,[],media.breitbart.com,"[media, newsmax]"
5,www.breitbart.com,[],media.breitbart.com,"[media, newsmax]"
6,www.breitbart.com,"[london, blair, backs, may, stay, brexit, defe...",media.breitbart.com,"[media, newsmax]"
7,dallas.texasguntrader.com,"[index, php]",dallas.texasguntrader.com,"[index, php]"
8,www.breitbart.com,[],media.breitbart.com,"[media, newsmax]"
9,www.breitbart.com,"[california, independent, autopsy, suggests, p...",media.breitbart.com,"[media, newsmax]"
